In [6]:
import re
import pandas as pd
import math
from datetime import datetime 

fecha = datetime.today().strftime('%Y%m%d')
num_format = re.compile("^[\-]?[0-9]*\.?[0-9]+$")

In [31]:
#importar datos desde archivo
df = pd.read_csv("D:\AEPS 2.0 Boyaca\datos\SOCIOECONOMICO_-principal\Procesado\BaseDatosSocioEcon.csv", encoding='ISO-8859-1', index_col=0, thousands=',')

In [21]:
##Funciones para acomodar latitud y longitud del archivo

def lati (x) :
    result = float(x)
    
    if result > 90.0  or result < -90.0 :
        digits = 1
        if  result < 0 :
            digits = int(math.log10(-result))
        else: 
            digits = int(math.log10(result))

        result = result / 10 ** digits

    return result

def longit (x) :
    result = float(x)
    
    if result > 180  or result < -180 :
        digits = 1
        if  result < 0 :
            digits = int(math.log10(-result))-1
        else: 
            digits = int(math.log10(result))-1
            result = -result

        result = result / 10 ** digits

    return result

In [5]:
#Se corrigen datos de latitud y longitud

df['L1_8'] = df['L1_8'].apply(lati)
df['L1_9'] = df['L1_9'].apply(longit)

In [6]:
municipios = {
    'MO':'MOTAVITA',
    'SA':'SAMACA',
    'SI':'SIACHOQUE',
    'SO':'SORACA',
    'TO':'TOCA',
    'TU':'TUNJA',
    'VE':'VENTAQUEMADA',
}

In [7]:
munici = df['L1_3'][df['L1_3'].isnull()] #datos con municipios vacios

ind = munici.index.tolist() #indices de los datos que no tienen municipio 

for i in ind:
    print('Agricultor ' + df.loc[i, 'L1_0'] + ' sin municipio')
    
    if df.loc[i, 'M1A_0'] :
        df.loc[i ,'L1_3'] = df.loc[i,'M1A_0']
        print('\tDato corregido con: ' + df.loc[i ,'L1_3'])
    elif df.loc[i, 'L1_0'] :
        df.loc[i ,'L1_3'] = municipios.get(df.loc[i, 'L1_0'].strip()[0:2], '')
        print('\tDato corregido con: ' + df.loc[i ,'L1_3'])
    else:
        print('\tNo se pudo completar dato')
    
    

Agricultor VE-FT-001 sin municipio
	Dato corregido con: VENTAQUEMADA
Agricultor VE-SU-001 sin municipio
	Dato corregido con: VENTAQUEMADA


In [8]:
#soc_people
#'kind_document'	'document'	'name'	'last_name'	'cellphone'	'sex'	'municipality'	'address'	'email'	'ext_id'

documents = df['L1_7']
names = df['L1_1']
last_names = df['L1_2']
cellphones = df['L1_6'].apply( lambda x: x[0:10]) #Se tomoa solo el primer numero
gene = df['M1A_2'].replace({"Hombre": 'M', "Mujer": 'F'}).fillna('O') #Si esta vacio por defecto se coloca O (Otro)
municipalityes = df['L1_3'].str.strip() #Se eliminan los espacios al inicio y final para evitar errores de realcion
address = df['L1_4']
codigos = df['L1_0']

d = {'kind_document':'N','document': documents, 'name': names,'last_name':last_names,
'cellphone': cellphones,'sex': gene, 'municipality': municipalityes,'address': address,	'email': '','ext_id': codigos}

soc_people = pd.DataFrame(d)

soc_people = soc_people.drop_duplicates() #Se eliminan duplicados


soc_people.to_csv('soc_people.csv',index=False)


,kind_document,document,name,last_name,cellphone,sex,municipality,address,email,ext_id
1,N,40011149,BLANCA MYRYAM,LOPEZ,3002694976,F,MOTAVITA,CARBONERA,,MO-CA-001
2,N,33365653,NIDIA R.,SAINEA P.,3106734360,F,MOTAVITA,CARBONERA,,MO-CA-002
3,N,23284299,ALVINA,ARCOS FONSECA,3124302859,F,MOTAVITA,CARBONERA,,MO-CA-005
4,N,1095592,ALFONSO,LOPEZ NIO,3133091070,M,MOTAVITA,CARBONERA,,MO-CA-006
5,N,4040556,NOE,MOZO,3132347019,M,MOTAVITA,CARBONERA,,MO-CA-009


In [107]:
#far_farms
#'name'	'latitude'	'location_comments'	'ext_id'	'farmer'	'longitude'

latitude_fram = df['L1_8']
longitude_fram = df['L1_9']
finca = 'Finca de ' + df['L1_1'] + ' ' + df['L1_2']
ext_id_fram = 'FINCA-' + df['L1_0'] #Se genera codigo de la finca con el codigo del agricultor

f = {'name': finca,'latitude': latitude_fram, 'location_comments': '','ext_id' : ext_id_fram,'farmer': codigos,'longitude': longitude_fram}

far_farms = pd.DataFrame(f)
far_farms = far_farms.drop_duplicates() #Se eliminan los duplicados

far_farms.to_csv('far_farms.csv', index=False)



,name,latitude,location_comments,ext_id,farmer,longitude
1,Finca de BLANCA MYRYAM LOPEZ,5.689167,,FINCA-MO-CA-001,MO-CA-001,-72.517222
2,Finca de NIDIA R. SAINEA P.,5.621111,,FINCA-MO-CA-002,MO-CA-002,-72.459167
3,Finca de ALVINA ARCOS FONSECA,5.681667,,FINCA-MO-CA-005,MO-CA-005,-72.343056
4,Finca de ALFONSO LOPEZ NIO,5.693889,,FINCA-MO-CA-006,MO-CA-006,-72.508333
5,Finca de NOE MOZO,5.586389,,FINCA-MO-CA-009,MO-CA-009,-72.545833


In [106]:
#far_plots
#'farm'	'name'	'latitude'	'altitude'	'ext_id'	'longitude'

ext_id_plot = 'LOTE-' + df['L1_0'] #Se genera codigo del lote con el codigo del agricultor
lote = 'Lote ' +  df['L1_0']

p = {'farm': ext_id_fram, 'name': lote, 'latitude': latitude_fram, 'altitude': 0, 'ext_id': ext_id_plot, 'longitude': longitude_fram}

far_plots = pd.DataFrame(p)
far_plots = far_plots.drop_duplicates() #Se eliminan los duplicados

far_plots.to_csv('far_plots.csv', index=False)

,farm,name,latitude,altitude,ext_id,longitude
1,FINCA-MO-CA-001,Lote MO-CA-001,5.689167,0,LOTE-MO-CA-001,-72.517222
2,FINCA-MO-CA-002,Lote MO-CA-002,5.621111,0,LOTE-MO-CA-002,-72.459167
3,FINCA-MO-CA-005,Lote MO-CA-005,5.681667,0,LOTE-MO-CA-005,-72.343056
4,FINCA-MO-CA-006,Lote MO-CA-006,5.693889,0,LOTE-MO-CA-006,-72.508333
5,FINCA-MO-CA-009,Lote MO-CA-009,5.586389,0,LOTE-MO-CA-009,-72.545833


In [6]:
#agregar consecutivos en los datos repetidos de la serie para diferenciar
def consecutive(x, serie):
    conse = 1
    for j in x:
        if conse != 1 :
            serie[j] = serie[j] + f'-{conse}'
            print(f'Dato modificado en el indice {j} con: ' + ext_id_event[j])
        conse += 1

In [7]:
def duplicado_consecutive(serie):
    duplic = serie[serie.duplicated(keep=False)] #Se verifica si hay duplicados en la serie

    if not duplic.empty :
        duplic = pd.DataFrame(duplic)

        duplic = duplic.groupby(duplic.columns.tolist()).apply(lambda x: x.index.tolist())
        
        duplic.apply(consecutive, args=(serie,))
    return serie

In [10]:
#far_production_events
# 'technical'	'ext_id'	'plot'	'form'

ext_id_event = 'EVENTO-SOC-' + fecha + '-' +df['L1_0'] #Se genera los id externos para los eventos productivos

ext_id_event = duplicado_consecutive(ext_id_event) #Se agrega un consecutivo a los id duplicados para diferenciar los eventos

df['event'] = ext_id_event #Se agregan los id de evento para futuras implementaciones

e = {'technical' : 1, 'ext_id' : ext_id_event, 'plot': ext_id_plot, 'form' : 1}

event = pd.DataFrame(e)

event.to_csv('far_production_events.csv', index=False)

Dato modificado en el indice 337 con: EVENTO-2021-02-SO-RS-009-2
1      EVENTO-2021-02-MO-CA-001
2      EVENTO-2021-02-MO-CA-002
3      EVENTO-2021-02-MO-CA-005
4      EVENTO-2021-02-MO-CA-006
5      EVENTO-2021-02-MO-CA-009
                 ...           
596    EVENTO-2021-02-VE-PP-010
597    EVENTO-2021-02-VE-PP-011
598    EVENTO-2021-02-VE-PP-012
599    EVENTO-2021-02-VE-FT-001
600    EVENTO-2021-02-VE-SU-001
Name: L1_0, Length: 600, dtype: object


In [71]:
#guardado preguntas
# far_responses_text   'event'	'raw_value'	'question'	'fixed_value'	'validated'
# far_responses_numeric 'event'	'raw_value'	'question'	'fixed_value'	'raw_units'	'fixed_units'	'validated'

def question (frame, event, question, value, tipo=''):
    #valida si tiene un tipo y si el valor es diferente de nan
    if tipo and value == value:
        #se verifica si es numero de no serlo se deja en cero
        isnumber = re.match(num_format,str(value)) 
        if not isnumber:
            value = 0

    frame = frame.append({'event': event, 'raw_value': value, 'question': question, 'fixed_value': value,'validated': 1}, ignore_index=True)
    return frame

In [ ]:

far_responses_text = pd.DataFrame(columns=['event','raw_value','question','fixed_value','validated'])
far_responses_numeric = pd.DataFrame(columns=['event','raw_value','question','fixed_value','raw_units','fixed_units','validated'])

for index, row in df.iterrows():

    print('Generando Preguntas del evento ' + row['event'] )

    #linea base demografica

    far_responses_text = question(far_responses_text, row['event'], "M1A_1", row['M1A_1'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_2", row['M1A_2'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "M1A_3", row['M1A_3'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_4", row['M1A_4'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_5", row['M1A_5'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_6", row['M1A_6'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_7", row['M1A_7'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_8", row['M1A_8'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "M1A_9", row['M1A_9'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "M1A_10", row['M1A_10'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "M1A_11", row['M1A_11'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "M1A_12", row['M1A_12'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_13", row['M1A_13'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_14", row['M1A_14'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_16", row['M1A_16'])
    far_responses_text = question(far_responses_text, row['event'], "M1A_17", row['M1A_17'])

    #Sistemas agricolas

    for i in range(16):

        far_responses_text = question(far_responses_text, row['event'], f'M1B_{i+1}', row[f'M1B_{i+1}'])
    
    
    #Eventos climaticos

    for i in range(11):

        far_responses_text = question(far_responses_text, row['event'], f'M2_{i+1}', row[f'M2_{i+1}'])

    
    #Servicios climaticos

    for i in range(17):

        far_responses_text = question(far_responses_text, row['event'], f'M3_{i+1}', row[f'M3_{i+1}'])


    #Seguridad alimentaria

    for i in range(12):

        far_responses_text = question(far_responses_text, row['event'], f'M4_{i+1}', row[f'M4_{i+1}'])

    
    #Servicios financieros

    for i in range(12):

        far_responses_text = question(far_responses_text, row['event'], f'M1D_{i+1}', row[f'M1D_{i+1}'])


    #Practicas ASAC

    k = 1

    while k < 19 :

        far_responses_numeric = question(far_responses_numeric, row['event'], f"M5_{k}", row[f'M5_{k}'], 'num')
        k += 1
        far_responses_text = question(far_responses_text, row['event'], f"M5_{k}", row[f'M5_{k}'])
        k += 1
        far_responses_text = question(far_responses_text, row['event'], f"M5_{k}", row[f'M5_{k}'])
        k += 1

far_responses_text.to_csv('far_responses_text.csv', index=False)
far_responses_numeric.to_csv('far_responses_numeric.csv', index=False)